# Low rank Affinity Matrix

In [ ]:
import numpy as np
import os
import pandas as pd
import string as str
import math
import sys
import time

from IPython.display import display, HTML
from sklearn.model_selection import KFold

Input your own path

In [ ]:
#thepath = os.getcwd().split('code_py_mec_optim\\')[0]
thepath = os.getcwd().split('Pycode_PC')[0]

data_X = pd.read_csv(thepath + "\\data_mec_optim\\marriage_personality-traits\\Xvals.csv", sep=',')
data_Y = pd.read_csv(thepath + "\\data_mec_optim\\marriage_personality-traits\\Yvals.csv", sep=',')

## Affinity Matrix Estimation

In [ ]:
sdX = data_X.std().values
sdY = data_Y.std().values
mX = data_X.mean().values
mY = data_Y.mean().values

Xvals = (data_X-mX)/sdX
Yvals = (data_Y-mY)/sdY

In [ ]:
sigma = 1
t_k = 0.3
l = 0.15

The loop is quicker than in R: it takes approximately 12 minutes against ~34min in R

In [ ]:
def affinity(Xvals, Yvals, l,s):

    ptm = time.time()
    
    phis = np.kron(Yvals.values.T, Xvals.values.T)
    dX = Xvals.shape[1]
    dY = Yvals.shape[1]
    n = Xvals.shape[0]
    
    p = np.repeat(1/n, n)
    q = np.repeat(1/n, n)

    f = np.repeat(1/n,n*n).reshape((n,n))
    g = np.repeat(1/n,n*n).reshape((n,n))

    pi_hat = np.identity(n)*(1/n)
    
    A = np.repeat(1,dX*dY)
    v = np.zeros(n)
    
    theval = 0
    theval_old = math.inf
    
    tolIpfp = 1e-8
    maxiterIpfp = 1e3
    iterCount = 0
    maxiter = 1e3
    tol = 1e-6

    while(np.abs(theval_old-theval)>tol and iterCount < maxiter):

        theval_old = theval

        Phi = Xvals.values.dot(A.reshape((dX,dY), order = 'F')).dot(Yvals.values.T)

        contIpfp = True
        iterIpfp = 0

        while(contIpfp):
            iterIpfp = iterIpfp + 1
            u = sigma * np.log(np.sum(np.multiply(g,np.exp((Phi - np.tile(v, n).reshape((n,n)))/sigma)), axis = 1))
            vnext = sigma * np.log(np.sum(np.multiply(f, np.exp((Phi - np.tile(u, n).reshape((n,n), order='F'))/sigma)), axis = 0))
            error = np.max(np.abs(np.sum(np.multiply(g,np.exp((Phi - np.tile(vnext, n).reshape((n,n)) - np.tile(u, n).reshape((n,n), order='F'))/sigma)), axis = 1) -1))

            #print(error)
            contIpfp = False if (error < tolIpfp) or (iterIpfp > maxiterIpfp) else True

            v = vnext

        fg = np.multiply(f,g)
        pi = np.multiply(fg,np.exp((Phi - np.tile(v, n).reshape((n,n)) - np.tile(u, n).reshape((n,n), order='F'))/sigma))

        thegrad = phis.dot((pi-pi_hat).flatten(order = 'F'))

        A = A - t_k*thegrad

        if l > 0:

            U,S,V = np.linalg.svd(A.reshape(dX,dY).T)

            D = [max(x - l*t_k, 0.0) for x in S]

            S = np.identity(len(D))
            np.fill_diagonal(S,D)

            A = U.dot(S.dot(V)).flatten(order = 'F')

        if l > 0:
            theval = np.sum(np.multiply(thegrad, A))- sigma* np.sum(np.multiply(pi,np.log(pi)))+l*np.sum(D)
        else:
            theval = np.sum(np.multiply(thegrad, A))- sigma* np.sum(np.multiply(pi,np.log(pi)))

        iterCount = iterCount + 1

        print("Number of iterations is:", iterCount)
        print("Value is:", theval)

    diff = time.time() - ptm
    print('Time elapsed = ', diff//60, 'minutes', diff%60, 'seconds')
    
    return A

In [ ]:
def IPFP(Xvals, Yvals, A, sigma):
    
    phis = np.kron(Yvals.values.T, Xvals.values.T)
    dX = Xvals.shape[1]
    dY = Yvals.shape[1]
    n = Xvals.shape[0]
    
    p = np.repeat(1/n, n)
    q = np.repeat(1/n, n)

    f = np.repeat(1/n,n*n).reshape((n,n))
    g = np.repeat(1/n,n*n).reshape((n,n))

    pi_hat = np.identity(n)*(1/n)
    
    v = np.zeros(n)
    
    tolIpfp = 1e-8
    maxiterIpfp = 1e3
    
    Phi = Xvals.values.dot(A.reshape((dX,dY), order = 'F')).dot(Yvals.values.T)

    contIpfp = True
    iterIpfp = 0
        
    while(contIpfp):
        iterIpfp = iterIpfp + 1
        u = sigma * np.log(np.sum(np.multiply(g,np.exp((Phi - np.tile(v, n).reshape((n,n)))/sigma)), axis = 1))
        vnext = sigma * np.log(np.sum(np.multiply(f, np.exp((Phi - np.tile(u, n).reshape((n,n), order='F'))/sigma)), axis = 0))
        error = np.max(np.abs(np.sum(np.multiply(g,np.exp((Phi - np.tile(vnext, n).reshape((n,n)) - np.tile(u, n).reshape((n,n), order='F'))/sigma)), axis = 1) -1))

        #print(error)
        contIpfp = False if (error < tolIpfp) or (iterIpfp > maxiterIpfp) else True

        v = vnext

    fg = np.multiply(f,g)
    pi = np.multiply(fg,np.exp((Phi - np.tile(v, n).reshape((n,n)) - np.tile(u, n).reshape((n,n), order='F'))/sigma))

    thegrad = phis.dot((pi-pi_hat).flatten(order = 'F'))
    fro = np.linalg.norm(thegrad)
    
    theval= np.sum(np.multiply(thegrad, A))- sigma* np.sum(np.multiply(pi,np.log(pi)))
    
    return theval, fro
    

In [ ]:
res = affinity(Xvals, Yvals, l,sigma)

## Cross Validation

In [ ]:
nfold = 5
nrep  = 5 
n = Xvals.shape[0]

nb_l = 11

lambda_list = np.arange(0.,0.55,0.05)
error_array = np.repeat(0.0, nb_l*nfold*nrep).reshape(nb_l, nfold*nrep)
fro_array   = np.repeat(0.0, nb_l*nfold*nrep).reshape(nb_l, nfold*nrep)

Warning: the loop below takes several hours to run

In [ ]:
for experiment in range(nrep):
    
    print("experiment is", experiment)
    kf = KFold(n_splits = nfold, shuffle = True)
    
    for j in range(nb_l):
        
        lambda_val = lambda_list[j]
        print("lambda is:",lambda_val)
        
        i = 0
        for index, indices in kf.split(list(range(nrep*(n//nrep)))):
            Xvals_test_cv = Xvals.loc[indices,:]
            Xvals_train_cv = Xvals[~Xvals.index.isin(indices)]

            Yvals_test_cv = Yvals.loc[indices,:]
            Yvals_train_cv = Yvals[~Yvals.index.isin(indices)]
            
            res_cv = affinity(Xvals_train_cv, Yvals_train_cv, lambda_val,sigma)
            
            theval, fro = IPFP(Xvals_train_cv, Yvals_train_cv,res_cv, sigma)
            
            error_array[j,nfold*experiment+i] = theval
            fro_array[j,nfold*experiment+i] = fro    
        

In [ ]:
mean_error = np.mean(error_array,axis = 1)
std_error  = np.std(error_array,axis =1, ddof = 1) / np.sqrt(nfold*nrep)

mean_fro = np.mean(fro_array_array,axis = 1)
std_fro  = np.std(fro_array,axis =1, ddof = 1) / np.sqrt(nfold*nrep)